In [1]:
import graphlab

In [2]:
from IPython.display import display

In [3]:
from IPython.display import Image

In [4]:
graphlab.canvas.set_target('ipynb')

In [201]:
data = graphlab.SFrame.read_csv('cleaned.csv', column_type_hints={'film_name': str,'empl':str})

Finished parsing file /Users/jmolayem/Desktop/cleaned.csv

Parsing completed. Parsed 100 lines in 0.41301 secs.

Finished parsing file /Users/jmolayem/Desktop/cleaned.csv

Parsing completed. Parsed 324352 lines in 0.375143 secs.

In [203]:
data.rename({'empl':'actor_name'})

id,actor_name,character,film_name
0,9901,Adam Riley,600353
1,9901,Captain' Robert Jackson,900984
2,9901,Jacob Krause,104113
3,9901,Carissa,104113
4,9901,Commander Fredericks,103934
5,9901,Peter McCollum,104180
6,9901,Charles Baker,104558
7,9901,Global Chairman,104552
8,9901,President of the UnitedStates ...,104636
9,9901,Audrey Thomas,600353


In [204]:
actors = data['actor_name'].unique()
films = data['film_name'].unique()

g = graphlab.SGraph()
g = g.add_edges(data, src_field='actor_name', dst_field='film_name')
g = g.add_edges(data, src_field='film_name', dst_field='actor_name')

print "Movie graph summary:\n", g.summary(), "\n"

Movie graph summary:
{'num_edges': 648704, 'num_vertices': 1614} 



In [207]:
print "Employee vertex sample:"
g.get_vertices(ids=actors).tail(5)

Employee vertex sample:


__id
8449
9120
9219
9364
9718


In [183]:
cc = graphlab.connected_components.create(g, verbose=False)
cc_out = cc['component_id']
print "Connected components summary:\n", cc.summary()

Connected components summary:
Class                                   : ConnectedComponentsModel

Graph
-----
num_edges                               : 648704
num_vertices                            : 1614

Results
-------
graph                                   : SGraph. See m['graph']
component size                          : SFrame. See m['component_size']
number of connected components          : 1
vertex component id                     : SFrame. See m['componentid']

Metrics
-------
training time (secs)                    : 0.2497

Queryable Fields
----------------
graph                                   : A new SGraph with the color id as a vertex property
component_id                            : An SFrame with each vertex's component id
component_size                          : An SFrame with the size of each component
training_time                           : Total training time of the model

None


In [184]:
cc_size = cc['component_size'].sort('Count', ascending=False)
cc_size

component_id,Count
128,1614


In [190]:
big_label = cc_size['component_id'][0]
big_names = cc_out[cc_out['component_id'] == big_label]
mainstream_actors = big_names.filter_by(actors, column_name='__id')['__id']

In [208]:
bacon_films = g.get_edges(src_ids=['10181'])

subgraph = graphlab.SGraph()
subgraph = subgraph.add_edges(bacon_films, src_field='__src_id',
                              dst_field='__dst_id')
subgraph.show(vlabel='id', highlight=['10181'])

In [209]:
subgraph = graphlab.SGraph()

for f in bacon_films['__dst_id']:
    subgraph = subgraph.add_edges(g.get_edges(src_ids=[f], dst_ids=None),
                                  src_field='__src_id', dst_field='__dst_id')
    
subgraph.show(highlight=list(bacon_films['__dst_id']), vlabel='__id', vlabel_hover=True)

In [210]:
def count_in_degree(src, edge, dst):
    dst['in_degree'] += 1
    return (src, edge, dst)

def get_degree(g):
    new_g = graphlab.SGraph(g.vertices, g.edges)
    new_g.vertices['in_degree'] = 0
    return new_g.triple_apply(count_in_degree, ['in_degree']).get_vertices()

degree = get_degree(g)

In [211]:
comparisons = ['10181', '9901']
degree.filter_by(comparisons, '__id').sort('in_degree', ascending=False)

__id,in_degree
9901,393
10181,97


In [212]:
actor_degree = degree.filter_by(actors, '__id')
actor_degree['in_degree'].show()

In [213]:
actor_degree.topk('in_degree')

__id,in_degree
6428,5082
7729,4789
8031,4655
8231,4608
8562,3993
8182,3991
7984,3771
6519,3742
5002,3713
8257,3699
